<a href="https://colab.research.google.com/github/randhirdinesh/C-PROGRAMMING/blob/main/pjrt25_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision timm opencv-python numpy matplotlib albumentations transformers tqdm


In [8]:
!pip uninstall albumentations transformers -y
!pip install albumentations transformers




Found existing installation: albumentations 1.4.20
Uninstalling albumentations-1.4.20:
  Successfully uninstalled albumentations-1.4.20
Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import timm
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoImageProcessor
from PIL import Image


In [ ]:
# Define Data Transformations
transform = A.Compose([
    A.Resize(256, 256),  # (Swin, ViTs, DINO), which work well with 256×256
    A.Normalize(mean=(0.5,), std=(0.5,)),  # ensures images have zero mean and unit variance
    ToTensorV2()  # Converts numpy to pytorch tensor
])

# Edge-Aware Filtering
def edge_detection(image):
    """Apply Canny Edge Detection"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    canny_edges = cv2.Canny(gray, 100, 200)  # Otsu's method can be used for adaptive thresholding
    return np.stack([gray, canny_edges], axis=-1)  # Stack edges & grayscale for richer feature learning

# Custom Dataset Class
class RustDataset(Dataset):
    def __init__(self, rgb_paths, thermal_paths, labels):
        self.rgb_paths = rgb_paths
        self.thermal_paths = thermal_paths
        self.labels = labels

        # Image Transformations
        self.transform = A.Compose([
            A.LongestMaxSize(640),
            A.PadIfNeeded(640, 640, border_mode=cv2.BORDER_CONSTANT, value=0),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])

    def edge_detection(self, image):
        """Apply Canny + Sobel Edge Detection"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        canny_edges = cv2.Canny(gray, 100, 200)

        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        sobel_combined = cv2.convertScaleAbs(cv2.magnitude(sobel_x, sobel_y))

        return np.stack([canny_edges, sobel_combined], axis=-1)

    def __len__(self):
        return len(self.rgb_paths)

    def __getitem__(self, idx):
        # Load images
        rgb_image = cv2.imread(self.rgb_paths[idx])
        thermal_image = cv2.imread(self.thermal_paths[idx])
        edge_image = self.edge_detection(rgb_image)

        # Apply transformations
        rgb_image = self.transform(image=rgb_image)['image']
        thermal_image = self.transform(image=thermal_image)['image']
        edge_image = torch.tensor(edge_image, dtype=torch.float32).permute(2, 0, 1) / 255.0  # Normalize to [0,1]

        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        return rgb_image, thermal_image, edge_image, label


✅ Hybrid Model (RGB + Thermal + Edge Features)
✅ Swin Transformer Backbone (State-of-the-Art for Image Processing)
✅ Self-Supervised Learning (DINO) for Feature Extraction
✅ Edge-Aware Processing (Canny & Sobel)
✅ Efficient Mixed-Precision Training (AMP)
✅ Fast DataLoader with Multi-Processing

In [ ]:
class RustDetectionModel(nn.Module):
    def __init__(self):
        super(RustDetectionModel, self).__init__()
        self.rgb_encoder = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=512)
        self.thermal_encoder = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=512)

        self.edge_conv = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.fc = nn.Sequential(
            nn.Linear(512 * 2 + 128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, rgb, thermal, edge):
        rgb_features = self.rgb_encoder(rgb)
        thermal_features = self.thermal_encoder(thermal)
        edge_features = self.edge_conv(edge).squeeze()
        combined = torch.cat([rgb_features, thermal_features, edge_features], dim=1)
        output = self.fc(combined)
        return output

model = RustDetectionModel().cuda()



📌 How to Improve?
✅ Replace CNN with Holistically-Nested Edge Detection (HED) for sharper edges.
✅ Try Dilated Convolutions to retain more details in rust boundaries.
✅ Use Dropout (0.3) to prevent overfitting.
✅ Replace Sigmoid with Softmax if you plan to detect multiple rust severity levels.

In [5]:
!pip install tensorboard


In [6]:
from torch.utils.tensorboard import SummaryWriter
import datetime

TRAINING ⚓

In [ ]:
def train_model(model, dataloader, epochs=10, lr=0.0001):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.BCELoss()
    model.train()

    # Initialize TensorBoard Writer
    log_dir = "runs/rust_detection_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    writer = SummaryWriter(log_dir)

    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        for i, (rgb, thermal, edge, label) in enumerate(progress_bar):
            rgb, thermal, edge, label = rgb.cuda(), thermal.cuda(), edge.cuda(), label.cuda()

            optimizer.zero_grad()
            output = model(rgb, thermal, edge).squeeze()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / len(dataloader))

            # Log loss to TensorBoard
            writer.add_scalar("Training Loss", loss.item(), epoch * len(dataloader) + i)

        # Log average loss per epoch
        writer.add_scalar("Average Loss per Epoch", total_loss / len(dataloader), epoch)

    writer.close()  # Close TensorBoard writer
    print("Training Complete.")


In [ ]:
train_model(model, train_dataloader, epochs=20)


In [7]:
tensorboard --logdir=runs


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (<ipython-input-7-8dfe2aebc706>, line 1)

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    iou_scores = []
    dice_scores = []

    with torch.no_grad():
        for rgb, thermal, edge, label in tqdm(dataloader, desc="Evaluating"):
            rgb, thermal, edge, label = rgb.cuda(), thermal.cuda(), edge.cuda(), label.cuda()
            output = model(rgb, thermal, edge).squeeze()
            pred = (output > 0.5).float()  # Convert to binary classification

            iou_scores.append(iou_score(pred, label).cpu().numpy())
            dice_scores.append(dice_score(pred, label).cpu().numpy())

    print(f"Average IoU Score: {np.mean(iou_scores):.4f}")
    print(f"Average Dice Score: {np.mean(dice_scores):.4f}")


In [ ]:
# Example image paths and labels
train_rgb_paths = ["data/rgb1.jpg", "data/rgb2.jpg"]
train_thermal_paths = ["data/thermal1.jpg", "data/thermal2.jpg"]
train_labels = [1, 0]  # 1 = Rust, 0 = No Rust

# Create Dataset & DataLoader
train_dataset = RustDataset(train_rgb_paths, train_thermal_paths, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

# Train Model
train_model(model, train_dataloader, epochs=10)
